In [1]:
import zarr
import xarray as xr
import time
import dask

In [2]:
ask_workers=2

In [3]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask.dataframe as dd

cluster = SLURMCluster(cores=1, memory='64GB',job_extra=['--partition=prepost','--account=yrf@cpu',
                                  '--ntasks=1','--cpus-per-task=1'], project='PerfTestPangeo', walltime='01:30:00')


/gpfswork/rech/yrf/rote001/conda/perf-pangeo/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39640 instead
  http_address["port"], self.http_server.port


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -A PerfTestPangeo
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=60G
#SBATCH -t 01:30:00
#SBATCH --partition=prepost
#SBATCH --account=yrf@cpu
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1

JOB_ID=${SLURM_JOB_ID%;*}

/gpfswork/rech/yrf/rote001/conda/perf-pangeo/bin/python -m distributed.cli.dask_worker tcp://130.84.132.27:34921 --nthreads 1 --memory-limit 64.00GB --name name --nanny --death-timeout 60



In [4]:
cluster.scale(ask_workers)

c = Client(cluster)

c

Client Scheduler: tcp://130.84.132.27:34921 Dashboard: http://130.84.132.27:39640/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
!squeue -u rote001
from dask.utils import ensure_dict, format_bytes
workers = c.scheduler_info()["workers"]
text="Workers= " + str(len(workers))
memory = [w["memory_limit"] for w in workers.values()]
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           2123737   prepost dask-wor  rote001 PD       0:00      1 (Priority)
           2123736   prepost dask-wor  rote001 PD       0:00      1 (Resources)
           2123941   prepost dask-wor  rote001  R       0:11      1 jean-zay-pp2
           2123942   prepost dask-wor  rote001  R       0:11      1 jean-zay-pp2
           2123620   prepost dask-wor  rote001  R      55:55      1 jean-zay-pp1
           2123621   prepost dask-wor  rote001  R      55:55      1 jean-zay-pp1
           2123335   prepost     srun  rote001  R    1:34:22      1 jean-zay-pp1
Workers= 2, Memory=128.00 GB


In [6]:
%time ds=xr.open_zarr('/gpfswork/rech/yrf/rote001/eNATL60/zarr/eNATL60-BLBT02-SSH-1h')

CPU times: user 84.4 ms, sys: 15 ms, total: 99.4 ms
Wall time: 109 ms


In [7]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 96.3 ms, sys: 2.36 ms, total: 98.7 ms
Wall time: 132 ms


In [ ]:
%time mean.load()